# Amazon OpenSearch hands-on lab

#### Original Workshop : https://catalog.us-east-1.prod.workshops.aws/workshops/f0213896-4dd9-494a-89c5-f7886b45ed4a/ko-KR

### 0. Setup Environment

In [2]:
#!pip install boto3

In [3]:
import boto3

### 1. OpenSearch 도메인 생성

In [4]:
import boto3
import time

def create_opensearch_domain(domain_name):
    client = boto3.client('opensearch')

    response = client.create_domain(
        DomainName=domain_name,
        EngineVersion='OpenSearch_1.3',
        ClusterConfig={
            'InstanceType': 'r6g.large.search',
            'InstanceCount': 2,
            'DedicatedMasterEnabled': False,
            'ZoneAwarenessEnabled': False,
        },
        EBSOptions={
            'EBSEnabled': True,
            'VolumeType': 'gp3',
            'VolumeSize': 100,
        },
        DomainEndpointOptions={
            'EnforceHTTPS': True,
            'TLSSecurityPolicy': 'Policy-Min-TLS-1-2-2019-07',
        },
        NodeToNodeEncryptionOptions={
            'Enabled': True,
        },
        EncryptionAtRestOptions={
            'Enabled': True,
        },
        AdvancedSecurityOptions={
            'Enabled': True,
            'InternalUserDatabaseEnabled': True,
            'MasterUserOptions': {
                'MasterUserName': 'admin',
                'MasterUserPassword': 'Passw0rd!'
            }
        },
    )

    print(response)

    endpoint_url = wait_for_domain_creation(client, domain_name)

    return endpoint_url


def wait_for_domain_creation(client, domain_name, wait_period=60, retries=30):
    """
    OpenSearch 도메인이 활성화될 때까지 기다린 후 도메인 엔드포인트 URL 반환

    :param client: boto3 OpenSearch client
    :param domain_name: OpenSearch 도메인 이름
    :param wait_period: 각 시도 사이에 대기할 시간 (초)
    :param retries: 최대 시도 횟수
    :return: 도메인 엔드포인트 URL
    """
    for _ in range(retries):
        describe_response = client.describe_domain(DomainName=domain_name)
        if 'Endpoint' in describe_response['DomainStatus']:
            return describe_response['DomainStatus']['Endpoint']
        time.sleep(wait_period)
    raise Exception("Domain creation timeout")

In [6]:
# OpenSearch 도메인 생성 (약 15분 소요)

domain_name = 'my-opensearch-domain1'
endpoint_url = create_opensearch_domain(domain_name)

print("ENDPOINT URL : ", endpoint_url)

{'ResponseMetadata': {'RequestId': '161af059-36e1-4d3e-ae7d-90b140f5d4e7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '161af059-36e1-4d3e-ae7d-90b140f5d4e7', 'content-type': 'application/json', 'content-length': '2293', 'date': 'Mon, 12 Jun 2023 07:23:55 GMT'}, 'RetryAttempts': 0}, 'DomainStatus': {'DomainId': '376278017302/my-opensearch-domain1', 'DomainName': 'my-opensearch-domain1', 'ARN': 'arn:aws:es:us-east-1:376278017302:domain/my-opensearch-domain1', 'Created': True, 'Deleted': False, 'Processing': True, 'UpgradeProcessing': False, 'EngineVersion': 'OpenSearch_1.3', 'ClusterConfig': {'InstanceType': 'r6g.large.search', 'InstanceCount': 2, 'DedicatedMasterEnabled': False, 'ZoneAwarenessEnabled': False, 'WarmEnabled': False, 'ColdStorageOptions': {'Enabled': False}}, 'EBSOptions': {'EBSEnabled': True, 'VolumeType': 'gp3', 'VolumeSize': 100, 'Iops': 3000, 'Throughput': 125}, 'AccessPolicies': '', 'SnapshotOptions': {'AutomatedSnapshotStartHour': 0}, 'CognitoOptions'

### 2. 도메인 보안 설정 업데이트 (AWS Console에서 작업)

<b>생성한 도메인 보안 설정을 수정합니다.</b><br>
새로운 탭에서 [Amazon OpenSearch 콘솔](https://us-east-1.console.aws.amazon.com/aos/home?#opensearch/domains)을 엽니다.<br>

생성된 도메인의 이름을 선택 합니다.<br>
![](./img/os-edit-security-config1.png)

화면 우측 상단의 <b>Actions</b>에서 Edit security configuration을 선택 합니다.<br>
![](./img/os-edit-security-config2.png)

<b>Access Policy</b>에서 Domain access policy 부분에서 <b>Only use fine-grained access control</b>을 선택한 뒤, 페이지 맨 하단의 <b>Save Changes</b>를 클릭 합니다.
![](./img/os-edit-security-config3.png)